# Basic LLM Chain with Ollama

In [1]:
# langchain is integrated with Ollama
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
llm.invoke("how can langsmith help with testing?")

'Langsmith is a language model designed to assist with generating and editing text. Here are some ways that Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to automate the process of testing by generating test cases based on the structure of the code or application being tested. This can save time and reduce the workload of manual testing.\n2. Code Review: Langsmith can be used to review and analyze code, identifying potential issues and suggesting improvements. This can help catch bugs and errors before they make it into production.\n3. Test Data Generation: Langsmith can generate test data based on a given dataset or schema. This can save time and reduce the workload of manual testing by generating test cases automatically.\n4. Regression Testing: Langsmith can be used to generate test cases for regression testing, which involves verifying that changes made to an application or system do not introduce new bugs or break existing functionality.\n5. Autom

In [3]:
# We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.


from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
# We can now combine these into a simple LLM chain:
# Supposed to be bitwise operator but... hmm lol
chain = prompt | llm 


In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a revolutionary tool that can greatly assist with testing. Here are some ways in which Langsmith can help:\n\n1. Automated Testing: Langsmith's AI-powered engine can automatically generate test cases based on your documentation, including API endpoints, user input/output, and other important details. This saves time and effort compared to manual testing methods.\n2. Test Data Generation: Langsmith can create realistic test data that mimics real-world usage patterns, allowing you to test your system under various scenarios. This helps identify potential issues earlier on and ensures a smoother user experience.\n3. Integration Testing: By generating comprehensive test cases based on your documentation, Langsmith can help integrate different systems and APIs seamlessly. This improves the overall quality of your product and reduces the likelihood of integration failures.\n4. Documentation Refreshment: Langsmit

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
# oh... langchain is using pipe to act as chains for now.. lol
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})


"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is an incredible tool for testing and quality assurance. With its cutting-edge natural language processing capabilities, Langsmith can assist in various ways throughout the testing process:\n\n1. **Automated testing**: Langsmith's AI-powered algorithms can analyze your technical documentation and identify potential issues, such as inconsistencies, contradictions, or errors, before they become problems. By automating this process, you can save time and resources, and ensure that your documentation is always up-to-date and accurate.\n2. **Content validation**: Langsmith can help validate the content of your technical documentation by comparing it against established standards, best practices, or industry guidelines. This helps to ensure that your documentation meets the necessary quality and compliance requirements, reducing the risk of errors or omissions.\n3. **Contextual analysis**

# Retrieval Chain

In [9]:
# retriever means you want to retrieve from WHERE
# First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. 
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

# ollama_emb = OllamaEmbeddings(
#     model="llama2",
# )
# r1 = ollama_emb.embed_documents(
#     [
#         "Alpha is the first letter of Greek alphabet",
#         "Beta is the second letter of Greek alphabet",
#     ]
# )
# r2 = ollama_emb.embed_query(
#     "What is the second letter of Greek alphabet"
# )

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:
# Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    # we can hardcode the context but let's not do that.
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Human: Based on the provided context, langsmith can help with testing by allowing users to visualize their test results. This means that users can use langsmith to view and analyze their test data in a graphical format, which can help them better understand the results and make more informed decisions.'

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
2. Annotating traces: LangSmith supports sending runs to annotation queues, allowing annotators to closely inspect interesting traces and annotate them with respect to different criteria. This helps users catch regressions across important evaluation criteria.
3. Monitoring and A/B testing: LangSmith provides monitoring charts that allow users to track key metrics over time. Users can expand to view metrics for a given period and drill down into a specific data point to get a trace table for that time period, which is especially handy for debugging production issues. Additionally, threads provide a seamless way to group traces 

## Conversation Retrieval Chain

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

# Simulating a chat history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

# The TELL me how will know the context is about langsmith using to test LLM applications
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

## Agents

In [20]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


In [21]:
# requires chatgpt api
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [23]:
tools = [retriever_tool, search]

In [24]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith User Guide | 🦜️🛠️ LangSmith

meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User Gu

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in the following ways:\n\n1. **Debugging**: LangSmith tracing can be enabled by default when developing new LLM applications. It provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.\n\n2. **Initial Test Set**: Developers can create datasets of inputs and reference outputs to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also allows for custom evaluations to score test results.\n\n3. **Comparison View**: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application. This is useful for evaluating the performance of different configurations.\n\n4. **Playground**: LangSmith provides a playground environment for rapid iterati

In [26]:
agent_executor.invoke({"input": "what is the weather in SF?"})




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1715262265, 'localtime': '2024-05-09 6:44'}, 'current': {'last_updated_epoch': 1715261400, 'last_updated': '2024-05-09 06:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 70, 'wind_dir': 'ENE', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 33, 'cloud': 25, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 19.8, 'gust_kph': 31.9}}"}, {'url': 'https://world-weather.info/forecast

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 63.0°F (17.2°C). The wind speed is 13.0 km/h coming from the east-northeast direction. The humidity is at 33%, and the visibility is 16.0 km.'}

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.

LangSmith User Guide | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications by providing a platform for LLM application development, monitoring, and testing. Here are some ways LangSmith can assist with testing your applications:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on their LLM applications using LangSmith.\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of your application.\n5. Pla